<center><img src="../图片数据/logo.png" alt="Header" style="width: 800px;"/></center>

@Copyright (C): 2010-2019, Shenzhen Yahboom Tech  
@Author: Malloy.Yuan  
@Date: 2019-07-17 10:10:02  
@LastEditors: Malloy.Yuan  
@LastEditTime: 2019-09-17 17:54:19  

# 自动驾驶 - 训练模型

在这个笔记本中，我们将训练一个神经网络获取一个输入图像，并输出一组x, y值对应于一个目标。
我们将使用之前课程我们用过的PyTorch深度学习框架来训练ResNet18神经网络结构模型，用于识别道路路况从而实现自动驾驶。

In [ ]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np

### 下载和提取数据
> # 如果你是在收集数据的JetBot进行培训，可以跳过此步骤!

In [ ]:
!unzip -q road_following.zip

解压缩完后你就可以看到一个名为' ' dataset_all ' '的文件夹出现在文件浏览器中。

### 创建数据库实例

在这里，我们创建一个自定义的``torch.utils.data.Dataset``，它实现了``__len__``和``__getitem__`` 函数。该类负责加载图像并解析图像文件名中的x、y值。因为我们实现了``torch.utils.data.Dataset``类，我们可以使用所有的torch数据实用工具

我们在数据集中硬编码了一些转换(比如颜色抖动)。我们将随机水平翻转设置为可选的(如果您想遵循非对称路径，比如道路)

Jetbot是否遵循某种约定无关紧要，你可以启用flips来扩充数据集。

In [ ]:
def get_x(path):
    """Gets the x value from the image filename"""
    return (float(int(path[3:6])) - 50.0) / 50.0

def get_y(path):
    """Gets the y value from the image filename"""
    return (float(int(path[7:10])) - 50.0) / 50.0

class XYDataset(torch.utils.data.Dataset):
    
    def __init__(self, directory, random_hflips=False):
        self.directory = directory
        self.random_hflips = random_hflips
        self.image_paths = glob.glob(os.path.join(self.directory, '*.jpg'))
        self.color_jitter = transforms.ColorJitter(0.3, 0.3, 0.3, 0.3)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        
        image = PIL.Image.open(image_path)
        x = float(get_x(os.path.basename(image_path)))
        y = float(get_y(os.path.basename(image_path)))
        
        if float(np.random.rand(1)) > 0.5:
            image = transforms.functional.hflip(image)
            x = -x
        
        image = self.color_jitter(image)
        image = transforms.functional.resize(image, (224, 224))
        image = transforms.functional.to_tensor(image)
        image = image.numpy()[::-1].copy()
        image = torch.from_numpy(image)
        image = transforms.functional.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        return image, torch.tensor([x, y]).float()
    
dataset = XYDataset('dataset_xy', random_hflips=False)

### 将数据集分割为训练集和测试集
一旦我们读取dataset，我们将分割训练集和测试集中的数据集。在这个例子中，我们分割训练并测试了90%-10%。测试集将用于验证我们训练的模型的准确性。

In [ ]:
test_percent = 0.1
num_test = int(test_percent * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - num_test, num_test])

### 创建数据加载器来批量加载数据

我们使用``DataLoader`` 类批量加载数据，洗牌数据，并允许使用多个子进程。在本例中，我们使用数据批量的大小为64。批量大小将基于内存可用的GPU，它可以影响模型的准确性。

In [ ]:
# 训练集
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

# 测试集
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

### 定义神经网络模型

我们使用的ResNet-18模型基于PyTorch TorchVision

在一个叫做“转移学习”的过程中，我们可以将一个预先训练好的模型(对数百万张图像进行训练)重新用于一个可能可用数据少得多的新任务。


更多信息请访问ResNet-18 : https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py

更多关于转移学习的细节(需要科学上网): https://www.youtube.com/watch?v=yofjFQddwHE 

In [ ]:
model = models.resnet18(pretrained=True)

ResNet模型已完全连接(fc)的最终层与512作为``in_features``，我们将训练回归，因此``out_features``作为1
最后，我们将模型转移到GPU上执行

In [ ]:
model.fc = torch.nn.Linear(512, 2)
device = torch.device('cuda')
model = model.to(device)

# 训练回归:

我们训练了50个时代，如果有减少损失的情况发生，我们将保存最好的模型

In [ ]:
# NUM_EPOCHS = 70
NUM_EPOCHS = 50
BEST_MODEL_PATH = 'best_steering_model_xy.pth'
best_loss = 1e9

optimizer = optim.Adam(model.parameters())

for epoch in range(NUM_EPOCHS):
    
    model.train()
    train_loss = 0.0
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        train_loss += loss
        loss.backward()
        optimizer.step()
    train_loss /= len(train_loader)
    
    model.eval()
    test_loss = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        test_loss += loss
    test_loss /= len(test_loader)
    
    print('%f, %f' % (train_loss, test_loss))
    if test_loss < best_loss:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_loss = test_loss

一旦模型训练完成，它将生成``best_steering_model_xy.pth`` 文件，我们将在自动驾驶例程中使用该模型进行推理.